In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500

In [2]:
cd ..

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus


In [3]:
from hopkins_cleaner import HopkinsDataCleaner, HopkinsDataFull

In [4]:
self = HopkinsDataFull()

In [5]:
self.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/us_data_cleaner.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  axis=0)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/hopkins_cleaner.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)


In [6]:
df = self.data

In [7]:
df.head(2)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,running_total_deaths,running_total_deaths_prev_day,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank
0,Thailand,2020-01-22,Not Provided,2.0,Not Provided-Thailand,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,15.0,101.0,1.0,1.0
1,Thailand,2020-01-23,Not Provided,3.0,Not Provided-Thailand,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,15.0,101.0,2.0,2.0


In [8]:
df.loc[df['state_name'] == 'Virginia'][['state_and_country','date','first_case_state_rank','first_case_country_rank']]

,state_and_country,date,first_case_state_rank,first_case_country_rank
11556,VA-US,2020-01-22,NaN,1.0
11557,VA-US,2020-01-23,NaN,2.0
11558,VA-US,2020-01-24,NaN,3.0
11559,VA-US,2020-01-25,NaN,4.0
11560,VA-US,2020-01-26,NaN,5.0
11561,VA-US,2020-01-27,NaN,6.0
11562,VA-US,2020-01-28,NaN,7.0
11563,VA-US,2020-01-29,NaN,8.0
11564,VA-US,2020-01-30,NaN,9.0
11565,VA-US,2020-01-31,NaN,10.0


In [ ]:
df.loc[df['state_name'] == 'Virginia']

In [ ]:
df_final.to_csv('output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

In [ ]:
max_date = df_final['date'].max()

In [ ]:
yesterday = df_final.loc[df_final['date'] == max_date]
yesterday['confirmed_cases'].sum(), yesterday['confirmed_recoveries'].sum(), yesterday['confirmed_deaths'].sum()

In [ ]:
df_final['daily_diff_cases'].sum(), df_final['daily_diff_recoveries'].sum(), df_final['daily_diff_deaths'].sum()

In [ ]:
df_final.loc[df_final['province_state'] == 'WA'].tail(5)

In [ ]:
df_final.loc[df_final['province_state'] == 'VA'].tail(5)

In [ ]:
df_final.loc[df_final['province_state'] == 'CA'].tail(5)

## QA of Data 

In [ ]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['province_state'].value_counts()

In [ ]:
df_final.loc[df_final['daily_diff_cases'] < 0]

In [ ]:
df_final.loc[(df_final['daily_diff_cases'] < 0) &
      (df_final['province_state'] == 'CA')]

In [ ]:
df_final.loc[df_final['country_region'] == 'China'].head(200).sort_values('rank')

In [ ]:
df_final.country_region.value_counts()